Neste projeto, foi desenvolvido e analisado diferentes arquiteturas de redes neurais artificiais aplicadas à resolução de operações matemáticas básicas. Foram implementados três modelos utilizando funções de ativação distintas: ReLU, LeakyReLU e Tanh, com arquiteturas aprimoradas de quatro camadas densas de 128 neurônios cada, incorporando técnicas de regularização L2 e Dropout para prevenção de overfitting.

O treinamento dos modelos foi otimizado com o uso do otimizador Adam, configurado com uma taxa de aprendizado reduzida (0.0005) para garantir maior estabilidade no processo de aprendizado. A técnica de EarlyStopping foi utilizada para evitar sobreajuste, interrompendo o treinamento automaticamente em caso de estagnação da melhoria do erro de validação.

Os resultados obtidos foram analisados através de gráficos de evolução da Loss e da MAE, além de tabelas detalhadas comparando o desempenho de cada modelo em novos exemplos aleatórios. A escolha do melhor modelo foi feita de maneira automatizada, utilizando o menor valor de erro de validação como critério.

Este trabalho demonstrou a eficácia da utilização de técnicas modernas de otimização e arquitetura em redes neurais aplicadas a tarefas de regressão, além de reforçar a importância de práticas como regularização, validação cruzada e análise comparativa de resultados para a construção de modelos preditivos robustos.

In [4]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.9 MB/s eta 0:00:00


In [5]:
# Importação das bibliotecas necessárias
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tqdm.notebook import tqdm
# Instead of: from tensorflow.keras.metrics import get as get_metric
from tensorflow.keras import metrics # Import the 'metrics' module
from tensorflow.keras.saving import register_keras_serializable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os

In [6]:
# 1. Preparação dos Dados
# Definição da semente para garantir reprodutibilidade dos resultados
np.random.seed(50)

# Função para gerar, normalizar e dividir os dados
def prepare_data():
    # Definição da quantidade de exemplos que serão gerados
    n_samples = 20000

    # Geração de números aleatórios entre 0 e 10 para x1 e x2
    x1 = np.random.uniform(0, 10, n_samples)
    x2 = np.random.uniform(0, 10, n_samples)

    # Escolha aleatória da operação a ser realizada para cada par de números
    operations = np.random.choice(['+', '-', '*', '/'], size=n_samples)

    # Lista para armazenar os resultados das operações
    results = []

    # Realização das operações matemáticas
    for a, b, op in zip(x1, x2, operations):
        if op == '+':
            results.append(a + b)
        elif op == '-':
            results.append(a - b)
        elif op == '*':
            results.append(a * b)
        elif op == '/':
            if b == 0:  # Tratamento para evitar divisão por zero
                b = 1e-6
            results.append(a / b)

    # Mapeamento das operações para valores numéricos: + → 0, - → 1, * → 2, / → 3
    op_map = {'+': 0, '-': 1, '*': 2, '/': 3}
    operations_num = np.array([op_map[op] for op in operations])

    # Criação da matriz de entrada X combinando x1, x2 e o código da operação
    X = np.column_stack((x1, x2, operations_num))

    # Conversão dos resultados para um array numpy
    y = np.array(results)

    # Aplicação da normalização nos dados de entrada
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Divisão dos dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Retorno dos conjuntos preparados e do objeto scaler
    return X_train, X_test, y_train, y_test, scaler

# Chamada da função para gerar e preparar os dados
X_train, X_test, y_train, y_test, scaler = prepare_data()



# Impressão de exemplos dos dados de treino para conferência

def ver_exemplos_desnormalizados(X, y, scaler, n=5):
    # Pega os primeiros n exemplos
    exemplos_X = X[:n]
    exemplos_y = y[:n]

    # Inverte a normalização
    desnormalizados = scaler.inverse_transform(exemplos_X)

    # Mapear cada valor da operação para o índice mais próximo entre [0, 1, 2, 3]
    codigos_op = []
    for op in desnormalizados[:, 2]:
        indice = np.argmin(np.abs(op - np.array([0, 1, 2, 3])))
        codigos_op.append(indice)

    # Traduz os códigos para os símbolos das operações
    operacoes = {0: '+', 1: '-', 2: '*', 3: '/'}
    operacoes_real = [operacoes[cod] for cod in codigos_op]

# Monta a tabela
    df = pd.DataFrame({
        'Número 1': desnormalizados[:, 0],
        'Número 2': desnormalizados[:, 1],
        'Operação': operacoes_real,
        'Resultado (y)': exemplos_y
    })

    # Arredonda para 2 casas decimais
    df = df.round(2)

    return df

# Exibir os primeiros 5 exemplos reais de X_train
df_exemplos = ver_exemplos_desnormalizados(X_train, y_train, scaler, n=5)
display(df_exemplos)

,Número 1,Número 2,Operação,Resultado (y)
0,0.89,7.10,+,7.99
1,9.50,7.28,-,2.22
2,7.51,8.00,-,-0.49
3,5.27,4.81,+,10.08
4,2.38,4.59,-,-2.21


In [7]:
# 2. Arquitetura da Rede Neural


# Função para criar um modelo de rede neural MLP
def build_model(activation='swish', regularization=None):
    # Inicializar um modelo sequencial
    model = keras.Sequential()

    # Primeira camada densa
    model.add(layers.Dense(128, activation=activation, input_shape=(3,),
                           kernel_regularizer=regularization))
    model.add(layers.Dropout(0.1))

    # Segunda camada densa
    model.add(layers.Dense(128, activation=activation,
                           kernel_regularizer=regularization))
    model.add(layers.Dropout(0.1))

    # Terceira camada densa
    model.add(layers.Dense(128, activation=activation,
                           kernel_regularizer=regularization))
    model.add(layers.Dropout(0.1))

    # Quarta camada densa
    model.add(layers.Dense(128, activation=activation,
                           kernel_regularizer=regularization))
    model.add(layers.Dropout(0.1))

    # Camada de saída
    model.add(layers.Dense(1))

    # Compilação do modelo com learning_rate reduzido
    optimizer = keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

    return model

# Função para criar um modelo usando LeakyReLU
def build_model_leakyrelu(regularization=None):
    model = keras.Sequential()

    # Primeira camada densa + LeakyReLU
    model.add(layers.Dense(128, input_shape=(3,),
                           kernel_regularizer=regularization))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dropout(0.1))

    # Segunda camada densa + LeakyReLU
    model.add(layers.Dense(128, kernel_regularizer=regularization))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dropout(0.1))

    # Terceira camada densa + LeakyReLU
    model.add(layers.Dense(128, kernel_regularizer=regularization))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dropout(0.1))

    # Quarta camada densa + LeakyReLU
    model.add(layers.Dense(128, kernel_regularizer=regularization))
    model.add(layers.LeakyReLU(alpha=0.01))
    model.add(layers.Dropout(0.1))

    # Camada de saída
    model.add(layers.Dense(1))

    optimizer = keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    return model

# Cria os Modelos

# Modelo utilizando ativação Swish
model_relu = build_model(activation='swish')

# Modelo utilizando ativação LeakyReLU
model_leakyrelu = build_model_leakyrelu()

# Modelo utilizando ativação Tanh
model_tanh = build_model(activation='tanh')


# Exibi Resumo dos Modelos

print("Modelo com Swish:")
model_relu.summary()

print("\nModelo com LeakyReLU:")
model_leakyrelu.summary()

print("\nModelo com Tanh:")
model_tanh.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Modelo com Swish:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,177 (196.00 KB)

...

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,177 (196.00 KB)

 Trainable params: 50,177 (196.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# 3. Treinamento dos Modelos

# Cria a pasta para salvar os modelos treinados, caso ainda não exista
if not os.path.exists('modelos_salvos'):
    os.makedirs('modelos_salvos')


def train_model(model, model_name, X_train, y_train, X_val, y_val, epochs=200):


    print(f"\nTreinando modelo: {model_name}")

    # EarlyStopping automático do Keras
    early_stopping = EarlyStopping(
        monitor='val_loss',         # Monitora a perda de validação
        patience=20,               # Espera até 20 épocas sem melhora antes de parar
        restore_best_weights=True,  # Recupera os melhores pesos encontrados
        min_delta=0.0001,            # Pequena mudança já conta como progresso
        verbose=0
    )

    # ModelCheckpoint para salvar o melhor modelo automaticamente
    model_checkpoint = ModelCheckpoint(
        filepath=f'modelos_salvos/melhor_modelo_{model_name}.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=0
    )

    # Histórico manual para registrar métricas
    history = {'loss': [], 'val_loss': [], 'mae': [], 'val_mae': []}

    # Loop de treino controlado manualmente
    with tqdm(total=epochs, desc=f"Treinando {model_name}") as pbar:
        for epoch in range(epochs):
            # Treinamento para uma época apenas
            hist = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=1,
                batch_size=32,
                verbose=0,
                callbacks=[early_stopping, model_checkpoint]
            )

            # Atualiza o histórico
            history['loss'].append(hist.history['loss'][0])
            history['val_loss'].append(hist.history['val_loss'][0])
            history['mae'].append(hist.history['mae'][0])
            history['val_mae'].append(hist.history['val_mae'][0])

            # Atualizar a descrição da barra de progresso com a loss atual
            pbar.set_postfix({
                "loss": f"{hist.history['loss'][0]:.6f}",
                "val_loss": f"{hist.history['val_loss'][0]:.6f}"
            })
            pbar.update(1)

            # Verifica se EarlyStopping parou
            if early_stopping.stopped_epoch > 0:
                print(f"Early stopping ativado no modelo {model_name} na época {epoch+1}.")
                break

    # Retorna o histórico completo do treino
    return history

# Treina cada modelo separadamente
history_relu = train_model(model_relu, "ReLU", X_train, y_train, X_test, y_test, epochs=200)
history_leakyrelu = train_model(model_leakyrelu, "LeakyReLU", X_train, y_train, X_test, y_test, epochs=200)
history_tanh = train_model(model_tanh, "Tanh", X_train, y_train, X_test, y_test, epochs=200)

#Organização dos Históricos

# Dicionário agrupando os históricos dos três modelos
histories = {
    'ReLU': history_relu,
    'LeakyReLU': history_leakyrelu,
    'Tanh': history_tanh
}


# Função para Plotar Gráficos de Comparação

def plot_histories(histories, metric='loss'):
    plt.figure(figsize=(12, 6))

    for name, history in histories.items():
        plt.plot(history[metric], label=f'{name} {metric} treino')
        plt.plot(history['val_' + metric], label=f'{name} {metric} validação')

    plt.title(f'Comparação de {metric.upper()} entre os Modelos')
    plt.xlabel('Épocas')
    plt.ylabel(metric.upper())
    plt.legend()
    plt.grid(True)
    plt.show()


#  Plotando os Gráficos de Comparação

# Plotar comparação da Loss
plot_histories(histories, metric='loss')

# Plotar comparação da MAE
plot_histories(histories, metric='mae')


Treinando modelo: ReLU


Treinando ReLU:   0%|          | 0/200 [00:00<?, ?it/s]


Treinando modelo: LeakyReLU


Treinando LeakyReLU:   0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
# 4. Análise Dinâmica e Carregamento do Melhor Modelo

from tensorflow.keras.models import load_model

# Função para analisar o desempenho e carregar automaticamente o melhor modelo
def analyze_and_load_best_model(histories):
    val_losses = {}

    # Coletar o último valor de val_loss de cada modelo
    for name, history in histories.items():
        val_loss_final = history['val_loss'][-1]
        val_losses[name] = val_loss_final

    # Ordenar os modelos pelo menor val_loss
    sorted_models = sorted(val_losses.items(), key=lambda x: x[1])

    print("\nRanking dos Modelos baseado no menor VAL_LOSS:")
    for rank, (model_name, loss) in enumerate(sorted_models, start=1):
        print(f"{rank}º lugar: {model_name} (Val Loss Final: {loss:.5f})")

    # Identificar o melhor modelo
    best_model_name = sorted_models[0][0]
    best_model_path = f'modelos_salvos/melhor_modelo_{best_model_name}.keras'

    print(f"\nCarregando o melhor modelo salvo: {best_model_name}")

    # Carregar o modelo sem compilar
    best_model_loaded = load_model(best_model_path, compile=False)

    # Compilar manualmente depois de carregar
    best_model_loaded.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )

    return best_model_name, best_model_loaded

    histories = {
    'ReLU': history_relu,
    'LeakyReLU': history_leakyrelu,
    'Tanh': history_tanh
}

# Rodar a função para analisar e carregar o melhor modelo
nome_melhor_modelo, melhor_modelo = analyze_and_load_best_model(histories)

In [ ]:
# Testar 3 Modelos e Exibir Resultados em Tabela (com Porcentagem de Acerto)

import pandas as pd

# Função para testar modelos e gerar tabela de resultados
def testar_modelos_em_tabela(modelos, nomes_modelos, scaler, n_testes=2):
    # Gerar pares aleatórios de números entre 0 e 10
    np.random.seed(50)
    x1 = np.random.uniform(0, 10, n_testes)
    x2 = np.random.uniform(0, 10, n_testes)
    operations = np.random.choice(['+', '-', '*', '/'], size=n_testes)

    # Construir dados de entrada
    novos_dados = []
    respostas_corretas = []

    for a, b, op in zip(x1, x2, operations):
        op_code = {'+': 0, '-': 1, '*': 2, '/': 3}[op]

        if op == '/' and b == 0:
            b = 1e-6  # Tratar divisão por zero

        if op == '+':
            resultado = a + b
        elif op == '-':
            resultado = a - b
        elif op == '*':
            resultado = a * b
        elif op == '/':
            resultado = a / b

        novos_dados.append([a, b, op_code])
        respostas_corretas.append(resultado)

    novos_dados = np.array(novos_dados)
    respostas_corretas = np.array(respostas_corretas)

    # Normalizar os novos dados
    novos_dados_normalizados = scaler.transform(novos_dados)

    # Lista para armazenar resultados
    resultados = []

    # Testar cada modelo
    for modelo, nome in zip(modelos, nomes_modelos):
        predicoes = modelo.predict(novos_dados_normalizados)

        for i in range(len(novos_dados)):
            num1, num2, op_code = novos_dados[i]
            operacao_str = {0: '+', 1: '-', 2: '*', 3: '/'}[int(op_code)]
            resposta_correta = respostas_corretas[i]
            predicao = predicoes[i][0]

            # Cálculo do erro percentual
            if resposta_correta != 0:
                erro_percentual = abs(predicao - resposta_correta) / abs(resposta_correta) * 100
            else:
                erro_percentual = abs(predicao - resposta_correta) * 100  # Caso resposta correta seja 0

            acuracia_percentual = 100 - erro_percentual
            acuracia_percentual = max(0, acuracia_percentual)  # Limitar para não ficar negativo

            # Definir se acertou ou errou (considerar erro relativo dentro de 5%)
            if erro_percentual <= 5:
                resultado = "ACERTOU"
            else:
                resultado = "ERROU"

            resultados.append({
                'Modelo': nome,
                'Operação': f"{num1:.1f} {operacao_str} {num2:.1f}",
                'Resultado Correto': f"{resposta_correta:.3f}",
                'Predito': f"{predicao:.3f}",
                'Resultado': resultado,
                'Acurácia (%)': f"{acuracia_percentual:.2f}%"
            })

    # Criar DataFrame para exibir
    df_resultados = pd.DataFrame(resultados)
    return df_resultados

# Rodar a função para os 3 modelos
modelos = [model_relu, model_leakyrelu, model_tanh]
nomes_modelos = ["ReLU", "LeakyReLU", "Tanh"]

# Gerar a tabela de resultados
df_resultados = testar_modelos_em_tabela(modelos, nomes_modelos, scaler, n_testes=3)

# Exibir a tabela
display(df_resultados)

In [ ]:

def testar_operacao_manual_tabelado(modelo, scaler, tolerancia_percentual=5):
    """
    Função para entrada manual do usuário: dois valores e uma operação (+, -, *, /).
    Faz a predição usando o modelo carregado e exibe o resultado em formato de tabela.
    """
    # Entrada dos valores
    try:
        num1 = float(input("Digite o primeiro número: "))
        num2 = float(input("Digite o segundo número: "))
    except ValueError:
        print("Erro: Você deve digitar valores numéricos!")
        return

    # Escolha da operação
    operacao = input("Escolha a operação (+, -, *, /): ").strip()
    if operacao not in ['+', '-', '*', '/']:
        print("Erro: Operação inválida. Use apenas +, -, * ou /.")
        return

    # Codificação da operação
    op_code = {'+': 0, '-': 1, '*': 2, '/': 3}[operacao]

    # Tratamento especial para divisão por zero
    if operacao == '/' and num2 == 0:
        print("Atenção: Divisão por zero não é permitida. Ajustando para divisor muito pequeno.")
        num2 = 1e-6

    # Cálculo da resposta correta
    if operacao == '+':
        resposta_correta = num1 + num2
    elif operacao == '-':
        resposta_correta = num1 - num2
    elif operacao == '*':
        resposta_correta = num1 * num2
    elif operacao == '/':
        resposta_correta = num1 / num2

    # Prepara a entrada para o modelo
    entrada = np.array([[num1, num2, op_code]])
    entrada_normalizada = scaler.transform(entrada)

    # Faz a predição
    predicao = modelo.predict(entrada_normalizada)[0][0]

    # Avalia o erro relativo
    if resposta_correta != 0:
        erro_percentual = abs(predicao - resposta_correta) / abs(resposta_correta) * 100
    else:
        erro_percentual = abs(predicao - resposta_correta) * 100

    acuracia_percentual = max(0, 100 - erro_percentual)

    # Avalia se acertou ou errou (tolerância de 5%)
    if erro_percentual <= tolerancia_percentual:
        resultado = "ACERTOU"
    else:
        resultado = "ERROU"

    # Monta a tabela usando pandas
    df_resultado = pd.DataFrame({
        'Número 1': [num1],
        'Operação': [operacao],
        'Número 2': [num2],
        'Resultado Correto': [f"{resposta_correta:.6f}"],
        'Predição do Modelo': [f"{predicao:.6f}"],
        'Resultado Final': [resultado],
        'Acurácia (%)': [f"{acuracia_percentual:.2f}%"]
    })

    # Exibe a tabela
    print("\nResultado da operação:")
    display(df_resultado)


testar_operacao_manual_tabelado(melhor_modelo, scaler)